In [ ]:
import json
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np
import torch
%matplotlib inline

In [ ]:
def json_to_mask(json_name):
    im = cv2.imread("Dataset/images/"+json_name+".png")
    print(im.shape)
    f = open("Dataset/labels/"+json_name+".json")
    annotation = json.load(f)
    masks = np.zeros((len(annotation["shapes"]), im.shape[0], im.shape[1]))
    for i in range(len(annotation["shapes"])):
        points = [[int(point[0]), int(point[1])] for point in annotation["shapes"][i]["points"]]
        points = np.array(points)
        print("Points:", points)
        cv2.fillPoly(masks[i, :, :], pts=[points], color=((i + 1) * 10, 0, 0))
        print(np.mean(masks[i]))
        masks[i, :, :] = (masks[i, :, :] == (i + 1) * 10)
    return torch.from_numpy(masks)

#json_to_mask("N00271097")
        

In [ ]:
masks = json_to_mask("N00271097")
plt.imshow(masks[0])

bounding_boxes = torch.zeros((len(masks), 4), dtype=torch.float)
print(torch.mean(masks[0]))

for index, mask in enumerate(masks):
    x, y = torch.where(mask != 0.0)
    print(x.shape)
    print(y.shape)

    bounding_boxes[index, 0] = torch.min(x)
    bounding_boxes[index, 1] = torch.min(y)
    bounding_boxes[index, 2] = torch.max(x)
    bounding_boxes[index, 3] = torch.max(y)

print(bounding_boxes)

In [ ]:
#Now test the dataset
from utils.params import Params
import dataset

transforms = None
params = Params("hparams.yaml", "DEFAULT")
train_data = dataset.CloudsDataset(params, transforms=None, root="Dataset/",  split="train")
